# Generate files for End2You

Here we provide an example on how to generate the required label files and the `input_file.csv` for the AffWild database in order to run End2You.

The AffWild folder is structured as follows:

```
AffWild
|─── annotations
|   |─── train
|   |   |─── arousal
|   |   |   |─── 105.txt
|   |   |   |─── 106.txt
|   |   |   |─── ...
|   |   |─── valence
|   |   |   |─── 105.txt
|   |   |   |─── 106.txt
|   |   |   |─── ...
|   |─── test
|   |   |─── arousal
|   |   |─── valence
|─── bboxes
|─── landmarks
|─── videos
|   |─── train
|   |   |─── 105.avi
|   |   |─── 106.avi
|   |   |─── ...
|   |─── test
```

## Set Paths

In [13]:
import numpy as np
from pathlib import Path

root_dir = Path('/vol/deform/pt511/db/AffWild/')
save_path = Path('/vol/deform/pt511/db/AffWild/end2you_files')

split = 'test'

videos_dir = root_dir / 'videos' / split
annotations = root_dir / 'annotations'

arousal_path = annotations / split / 'arousal'
valence_path = annotations / split / 'valence'

## Write `input_file.csv`

In [14]:
from moviepy.editor import VideoFileClip

videos, annots = [], []
for video_path in list(videos_dir.glob('*')):
    
    video_name = video_path.name[:-4]
    clip = VideoFileClip(str(video_path))
    fps = np.ceil(clip.fps).astype(int)
    
    arousal_file = arousal_path / (video_name + '.txt')
    valence_file = valence_path / (video_name + '.txt')
    
    # Load arousal/valence annotation values
    arousal_annot = np.loadtxt(arousal_file, dtype=np.float32).reshape(-1,1)
    valence_annot = np.loadtxt(valence_file, dtype=np.float32).reshape(-1,1)
    annotations = np.hstack([arousal_annot, valence_annot])
    
    # Find the time of each frame based on FPS
    time = [0.0]
    for t in range(annotations.shape[0]):
        time.append(time[-1] + 1/clip.fps)
    time = np.array(time).reshape(-1,1)
    
    # Save time/arousal/valence for each video in a csv file
    data = np.hstack([time[:-1], annotations]).astype(np.float32)
    save_annot = save_path / 'video_annot' / (video_name + '.csv')
    np.savetxt(str(save_annot), data, fmt='%.4e', delimiter=',', header = 'time,arousal,valence')
    
    # Store videos and annotations paths to write `input_file.csv` file
    videos.append(str(video_path))
    annots.append(str(save_annot))

In [15]:
vs = np.array(videos).reshape(-1, 1)
annts = np.array(annots).reshape(-1, 1)

In [16]:
# Save `input_file.csv`
input_file_path = save_path / 'input_file_test.csv'
np.savetxt(str(input_file_path), np.hstack([vs, annts]), fmt='%s', delimiter=',', header='video_file,annotation_file')